## 在 Data_Cleaning.ipynb的基础上做的改变
1. 将72个20分钟段特征去掉,改成24个小时段+一个小时的3个20分钟段组合
2. 增加天气数据
 - 天气中风向的异常值99107用他前一个风向与后一个风向的平均值代替

1. 读取测试训练的t3,t4,t5,t6.t7,合并训练测试的t7生成天气数据
2. 按照20分钟时间窗口得到 t5_train,t6_train,t5_test_2h_ago,t6_testt_2h_ago
3. t5_test_2h_ago,t6_testt_2h_ago,时间窗口加2小时去掉标签,得到t5_test,t6_test
4. 按照时间日期给t5_train,t6_train,t5_test,t6_test,加特征是否假期,星期几,几点钟,在那个20分钟段
5. 按照时间日期增加天气数据并fillna
6. 清洗t4,t5得每条路线的,宽度长度岔路数,t5_train,t5_test按照Intersection 和 Tollgates 添加路途宽度长度岔路数特征
7. t5_train,t6_train,t5_test,t6_test数据格式修改到要求的格式,保存到csv文件中
8. 查看文件发现并修改一些问题

In [1]:
import pandas as pd 
import glob
t3 = pd.read_csv('dataSets/training/links (table 3).csv')
t4 = pd.read_csv('dataSets/training/routes (table 4).csv')
t5_train = pd.read_csv('dataSets/training/trajectories(table 5)_training.csv')
t6_train = pd.read_csv(glob.glob("dataSets/training/*.csv")[-1])
t6_test_2h_ago = pd.read_csv(glob.glob("dataSets/testing_phase1/*.csv")[0])
t5_test_2h_ago = pd.read_csv(glob.glob("dataSets/testing_phase1/*.csv")[-1])

In [2]:
# 
t7 = pd.read_csv(glob.glob("dataSets/training/*.csv")[0])
t71 =pd.read_csv(glob.glob("dataSets/testing_phase1/*.csv")[1])
t7 = t7.append(t71).reset_index()
del t71

In [3]:
# 修改风向
for i, row in t7.iterrows():
    if row['wind_direction']== 999017.0:
        last = t7.loc[i-1,'wind_direction']
        nest = t7.loc[i+1,'wind_direction']
        t7.loc[i, 'wind_direction'] = (last + nest)/2.0

In [4]:
t7.wind_direction.max()

360.0

In [5]:
# 按照时间窗口重组数据: http://stackoverflow.com/questions/36914892/python-how-to-group-pandas-data-frame-in-a-certain-time-window
t5_train['starting_time'] =  pd.to_datetime(t5_train['starting_time'] , format='%Y-%m-%d %H:%M:%S')
t5_train = t5_train.set_index(['starting_time'])

t5_train = t5_train.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
       .reset_index().rename(columns = {'travel_time':'avg_travel_time'})
    
t5_test_2h_ago['starting_time'] =  pd.to_datetime(t5_test_2h_ago['starting_time'] , format='%Y-%m-%d %H:%M:%S')
t5_test_2h_ago = t5_test_2h_ago.set_index(['starting_time'])

t5_test_2h_ago = t5_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
       .reset_index().rename(columns = {'travel_time':'avg_travel_time'})
    
    
t6_train['time'] =  pd.to_datetime(t6_train['time'] , format='%Y-%m-%d %H:%M:%S')
t6_train = t6_train.set_index(['time'])

# 车流量
t6_train = t6_train.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
       .reset_index().rename(columns = {0:'volume'})
    
    
t6_test_2h_ago['time'] =  pd.to_datetime(t6_test_2h_ago['time'] , format='%Y-%m-%d %H:%M:%S')
t6_test_2h_ago = t6_test_2h_ago.set_index(['time'])

# 车流量
t6_test_2h_ago = t6_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
       .reset_index().rename(columns = {0:'volume'})    

In [6]:
# apped 2h前测试数据
t5_train= t5_train.append(t5_test_2h_ago).reset_index()
t6_train= t6_train.append(t6_test_2h_ago).reset_index()
# 增加两小时作为测试数据
t5_test_2h_ago['starting_time'] = t5_test_2h_ago['starting_time'] + pd.DateOffset(hours=2)
t6_test_2h_ago['time'] = t6_test_2h_ago['time'] + pd.DateOffset(hours=2)
# 测试数据
t5_test,t6_test =t5_test_2h_ago,t6_test_2h_ago
del t5_test_2h_ago,t6_test_2h_ago

In [7]:
# 中秋国庆假期

from datetime import datetime
start1 = datetime(2016, 9, 15)
end1 = datetime(2016, 9, 17)
start2 = datetime(2016, 10, 1)
end2 = datetime(2016, 10, 7)
rng = pd.date_range(start1, end1).append(pd.date_range(start2, end2))

# 是否假期,星期,小时,在那个时间窗口

def generateDataTime(X,label):
    
    # 小时,星期,时间窗口
    hour = pd.get_dummies(X[label].dt.hour,prefix='hour_')
    weekday = pd.get_dummies(X[label].dt.weekday_name)
    minute= pd.get_dummies(X[label].dt.minute)
    # 添加到X
    X = pd.concat([X,weekday,hour, minute], axis=1)
    # 增加 date hour特征方便与t7配对以添加天气数据
    X['date']=X[label].dt.date
    X['date'] = X['date'].astype(str)
    X['date'] = pd.to_datetime(X['date'], format='%Y-%m-%d')
    X['hour']=X[label].dt.hour.astype(int)
    # 是否在假期内
    date = X[label].dt.date
    for i, row in X.iterrows():
        X.loc[i, "holiday"] = 0
        if date.loc[i] in rng: X.loc[i, "holiday"] = 1
    return X

In [8]:
# 生成包含日期数据的数据
t5_test = generateDataTime(t5_test,'starting_time')
t6_test = generateDataTime(t6_test,'time')
t5_train = generateDataTime(t5_train,'starting_time')
t6_train = generateDataTime(t6_train,'time')

## 按照日期时间添加天气数据

In [9]:
t7['date'] = pd.to_datetime(t7['date'], format='%Y-%m-%d')

In [10]:
# 将hour转变成3小时间隔,然后与天气数据组合
def add_weather(df):
    for i, row in df.iterrows():
        if row['hour'] in [23,0,1]: df.loc[i, "hour"] = 0
        elif row['hour'] in [2,3,4]: df.loc[i, "hour"] = 3 
        elif row['hour'] in [5,6,7]: df.loc[i, "hour"] = 6         
        elif row['hour'] in [8,9,10]: df.loc[i, "hour"] = 9         
        elif row['hour'] in [11,12,13]: df.loc[i, "hour"] = 12         
        elif row['hour'] in [14,15,16]: df.loc[i, "hour"] = 15         
        elif row['hour'] in [17,18,19]: df.loc[i, "hour"] = 18         
        elif row['hour'] in [20,21,22]: df.loc[i, "hour"] = 21
    return pd.merge(df,t7,on =['date', 'hour'] ,how='left')
 

In [11]:
t5_test = add_weather(t5_test)
t6_test = add_weather(t6_test)
t5_train =add_weather(t5_train)
t6_train = add_weather(t6_train)

In [24]:
# 删除不必要的列
t5_test = t5_test.drop(['hour','date'],axis=1)
t6_test = t6_test.drop(['hour','date'],axis=1)
t5_train =t5_train.drop(['index_x','hour','date','index_y'],axis=1)
t6_train = t6_train.drop(['index_x','hour','date','index_y'],axis=1)

In [25]:
# 道路信息聚合

# 将110,123 变成[110,123] 
def split(str): return str.split(",")
t4.link_seq = t4.link_seq.apply(split)

# 将数据按link_seq中各link展开
rows = []
_ = t4.apply(lambda row: [rows.append([row['intersection_id'], row['tollgate_id'], nn]) 
                         for nn in row.link_seq], axis=1)
col = ['intersection_id', 'tollgate_id','link_id']
t4_new = pd.DataFrame(rows, columns=col)

# pd.merge(t4_new,t3,on = 'link_id',how='left')
# 提取每条link的是否有in,out交叉路口
t3['cross_in'] = 0
t3['cross_out'] = 0

for i, row in t3.iterrows():
    if ',' in str(row['in_top']):
        t3.loc[i, "cross_in"] = 1
    if ',' in str(row['out_top']):
        t3.loc[i, "cross_out"] = 1
        
t3['link_id'] = t3['link_id'].astype(str)
t4_new['link_id'] = t4_new['link_id'].astype(str)
t4_new = pd.merge(t4_new,t3,on = 'link_id',how='left')
t4_new.drop(['in_top','out_top'],axis =1,inplace= True)

In [26]:
# 聚合t4_new
# in out 岔路口数
b1 = t4_new[['intersection_id', 'tollgate_id','cross_in']]\
           .groupby(['intersection_id', 'tollgate_id'])\
            .cross_in.sum().reset_index().rename(columns = {'cross_in':'in_link_cross_count'})
b2 = t4_new[['intersection_id', 'tollgate_id','cross_out']]\
        .groupby(['intersection_id', 'tollgate_id'])\
        .cross_out.sum().reset_index().rename(columns = {'cross_out':'out_link_cross_count'})
road = pd.merge(b1,b2,on =['intersection_id', 'tollgate_id'] ,how='left')

# 路程
b1 = t4_new[['intersection_id', 'tollgate_id','length']].groupby(['intersection_id', 'tollgate_id']).length.sum().reset_index()
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
## 车道数1车道2车道3车道4车道的link数，后期考虑：占总路程的比率
# 各个路径的link总数
b1 = t4_new[['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'link_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')

In [27]:
# 1,2,3,4车道道路长度
b1 = t4_new[t4_new.lanes== 1][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'1_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 2车道路长度
b1 = t4_new[t4_new.lanes== 2][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'2_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 3车道路长度
b1 = t4_new[t4_new.lanes== 3][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'3_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 4车道路长度
b1 = t4_new[t4_new.lanes== 4][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'4_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')


In [28]:
b1 = t4_new[t4_new.lanes== 1][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'1_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 2车道link数
b1 = t4_new[t4_new.lanes== 2][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'2_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 3车道link数
b1 = t4_new[t4_new.lanes== 3][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'3_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 4车道link数
b1 = t4_new[t4_new.lanes== 4][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'4_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')

road.fillna(0,inplace=True)


In [29]:
t5_test = pd.merge(t5_test,road,on =['intersection_id', 'tollgate_id'] ,how='left')
t5_train = pd.merge(t5_train,road,on =['intersection_id', 'tollgate_id'] ,how='left')

In [30]:
t5_test['end'] = t5_test['starting_time'] + pd.DateOffset(minutes=20)
t5_train['end'] = t5_train['starting_time'] + pd.DateOffset(minutes=20)
t6_test['end'] = t6_test['time'] + pd.DateOffset(minutes=20)
t6_train['end'] = t6_train['time'] + pd.DateOffset(minutes=20)


# http://stackoverflow.com/questions/30132282/datetime-to-string-with-series-in-python-pandas
def timewindows(X,start,end):
    s= X[start].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    e = X[end].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    X['time_window'] = '['+ s +','+ e +')'
    return X.drop([start,end],axis =1)


t5_test = timewindows(t5_test,'starting_time','end')
t5_train = timewindows(t5_train,'starting_time','end')
t6_test = timewindows(t6_test,'time','end')
t6_train = timewindows(t6_train,'time','end')

t5_test = t5_test.set_index(['intersection_id','tollgate_id','time_window'])
t5_train = t5_train.set_index(['intersection_id','tollgate_id','time_window'])
t6_test = t6_test.set_index(['tollgate_id','time_window', 'direction'])
t6_train = t6_train.set_index(['tollgate_id','time_window', 'direction'])

t5_testcol,t5_traincol = list(t5_test.columns.values),list(t5_train.columns.values)
t6_testcol,t6_traincol = list(t6_test.columns.values),list(t6_train.columns.values)
mis_5 =  [x for x in t5_traincol  if x not in t5_testcol]
mis_6 =  [x for x in t6_traincol  if x not in t6_testcol]

for label in mis_5:
    t5_test[label] = 0
for label in mis_6:
    t6_test[label] = 0
    
# 调整各列顺序
t5_test = t5_test[t5_traincol]
t6_test = t6_test[t6_traincol]    

In [31]:
def fill(df):
    return df.fillna(df.mean())
t5_test = fill(t5_test)
t6_test = fill(t6_test)
t5_train =fill(t5_train)
t6_train =fill(t6_train)

In [32]:
t5_test.to_csv('dataSets/big_testset_task1_version2.csv')
t5_train.to_csv('dataSets/big_trainset_task1_version2.csv')
t6_test.to_csv('dataSets/big_testset_task2_version2.csv')
t6_train.to_csv('dataSets/big_trainset_task2_version2.csv')